In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()


In [3]:
import os

os.getcwd()

'/home/jovyan'

In [4]:
df = spark.read.parquet('work/files/OriginaisNetflix - Python (1).parquet')


In [5]:
df.head()

Row(Title='House of Cards', Genre='Political drama', GenreLabels='political,drama', Premiere='1-Feb-13', Seasons='6 seasons, 73 episodes', SeasonsParsed=6, EpisodesParsed=73, Length='42–59 min.', MinLength=42, MaxLength=59, Status='Ended', Active=0, Table='Drama', Language='English', dt_inclusao='2021-03-16T21:20:24.167-03:00')

In [6]:
#Transform columns ["Premiere" ,"dt_inclusao"] to timestamp
df = df.withColumn("Premiere", (col("Premiere").cast("timestamp")))
df=  df.withColumn("dt_inclusao", (col("dt_inclusao").cast("timestamp")))

NameError: name 'col' is not defined

In [ ]:
df.printSchema()

In [ ]:
#Ordenar os dados por ativos e gênero de forma decrescente, 0 = inativo e 1 = ativo, todos com número 1 devem aparecer primeiro.
df= df.sort(col("Active").desc(),col("Genre").desc())

In [ ]:
df.show()

In [ ]:
#Remover linhas duplicadas e trocar o resultado das linhas que tiverem a coluna "Seasons" de "TBA" para "a ser anunciado".
from pyspark.sql.functions import when, col

#print shape dataframe
print(f'Dataframe before drop duplicates: {df.toPandas().shape}')
df = df.dropDuplicates()
print(f'Dataframe after drop duplicates {df.toPandas().shape}')
df = df.withColumn("Seasons", when(col("Seasons") == "TBA", "a ser anunciado").otherwise(col("Seasons")))
print(f'Number rows "A ser anunciado:" {df.filter(col("Seasons") == "a ser anunciado").count()}')

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
# Criar uma coluna nova chamada "Data de Alteração" e dentro dela um timestamp.
df = df.withColumn("Data de Alteração", (col("dt_inclusao").cast("timestamp")))

In [ ]:
df.printSchema()

In [ ]:
#Trocar os nomes das colunas de inglês para português, exemplo: "Title" para "Título" (com acentuação).
# columns : ['Título',  'Gênero',  'GenreLabels',  'Premiere',  'Seasons',  'SeasonsParsed',  'EpisodesParsed',  'Length',  'MinLength',  'MaxLength',  'Status',  'Active',  'Table',  'Language',  'dt_inclusao',  'Data de Alteração']
df = df.withColumnRenamed("Title", "Título")
df = df.withColumnRenamed("Genre", "Gênero")
df = df.withColumnRenamed("GenreLabels", "Labels de Gênero")
df = df.withColumnRenamed("Premiere", "Data de Lançamento")
df = df.withColumnRenamed("Seasons", "Temporada")
df = df.withColumnRenamed("SeasonsParsed", "Temporadas Parsadas")
df = df.withColumnRenamed("EpisodesParsed", "Episódios Parsados")
df = df.withColumnRenamed("Length", "Duração")
df = df.withColumnRenamed("MinLength", "Duração Mínima")
df = df.withColumnRenamed("MaxLength", "Duração Máxima")
df = df.withColumnRenamed("Status", "Status")
df = df.withColumnRenamed("Active", "Ativo")
df = df.withColumnRenamed("Table", "Tabela")
df = df.withColumnRenamed("Language", "Idioma")
df = df.withColumnRenamed("dt_inclusao", "Data de Inclusão")




In [ ]:
df.printSchema()

In [ ]:
#Criar apenas 1 .csv com as seguintes colunas que foram nomeadas anteriormente "Title, Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as colunas devem estar em português com header e separadas por ";".
df.toPandas().to_csv('work/files/Netflix - Python (1).csv', sep=';', encoding='utf-8', index=False)

In [9]:
import boto3

session = boto3.Session(
    aws_access_key_id='AWS_ACCESS_KEY_ID',
    aws_secret_access_key='AWS_SECRET_ACCESS_KEY',
)
s3 = session.resource('s3')
# Filename - File to upload
# Bucket - Bucket to upload to (the top level directory under AWS S3)
# Key - S3 object name (can contain subdirectories). If not specified then file_name is used
s3.meta.client.upload_file(Filename='work/files/Netflix - Python (1).csv', Bucket='confitec_prova', Key='s3_output_key')

ModuleNotFoundError: No module named 'boto3'